In [ ]:
import pandas as pd
import asyncio
import getpass
from planet import Auth, Session
import json
from planet_helpers import set_filters, parse_polygon
from datetime import datetime
from tqdm import tqdm
import os

DOWNLOAD_DIR = '../data/planet_data'
FILTER_DIR = '../data/filters'
SEARCH_DIR = '../data/searches'

In [ ]:
user = input("Username: ")
pw = getpass.getpass()
auth = Auth.from_login(user,pw)
auth.store()

In [ ]:
events = pd.read_csv('../data/ACLED_Ukraine_events_sample.csv')[['location_id', 'event_date', 'event_id_cnty', 'timeline_id']]
non_events = pd.read_csv('../data/ACLED_Ukraine_non_events_sample.csv')[['location_id', 'event_date', 'timeline_id']]

places = pd.read_csv("../data/places.csv")

# merge events with places on location_id
events = events.merge(places, on='location_id')
non_events = non_events.merge(places, on='location_id')

In [ ]:
all_filters = []

for event in tqdm(non_events.iterrows(), total=len(non_events)):
   
   coords = parse_polygon(event[1]['geometry'])
   
   geom = {
         "type": "Polygon",
         "coordinates": coords
      }

   date = datetime.strptime(event[1]["event_date"], "%Y-%m-%d")
   # subtract 5 days from the event date
   five_before = (date - pd.DateOffset(days=5)).to_pydatetime()
   five_after = (date + pd.DateOffset(days=5)).to_pydatetime()

   filters = set_filters(from_date=five_before, to_date = five_after, geom=geom)
   all_filters.append(filters)

# save filters
with open(FILTER_DIR + '/filters_non_events.jsonl', 'w') as file:
    for entry in all_filters:
        # Convert each dictionary to a JSON string and write it to the file
        file.write(json.dumps(entry) + '\n')

Apply the filter to searches

In [ ]:
event_type = 'non_events'
df = pd.read_csv('../data/ACLED_Ukraine_' + event_type + '_sample.csv')

async with Session() as sess:
    cl = sess.client('data')

    with open(FILTER_DIR + '/filters_' + event_type + '.jsonl', 'r') as file:
        # Initialize tqdm with manual update
        pbar = tqdm(total=len(df))
        
        for i, line in enumerate(file):
            timeline_id = str(df['timeline_id'][i])
            name = event_type + '_' + timeline_id
            JSON_DIR = SEARCH_DIR + '/' + name + '.json'

            pbar.update(1)
            
            # Check if file already exists
            if os.path.exists(JSON_DIR):
                # print(f"Search {name} already exists")
                continue
            else:
                # print(f"Creating search {name}")
                filters = json.loads(line)
                
                request = await cl.create_search(name=name,
                                                 search_filter=filters,
                                                 item_types=["PSScene"])
                with open(JSON_DIR, 'w') as f:
                    f.write(json.dumps(request))
        
        # Close the progress bar
        pbar.close()
